# Item-Based Collaborative Filtering

We'll be building a full blown recommender system that looks at the movie ratings every user gave in order to recommend them movies they are most likely to love.

In [ ]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('/Users/jacquesthibodeau/Desktop/Media/Online Courses/Coding courses/Data Science and Machine Learning with Python/Python Data Science course material/ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3))

m_cols = ['movie_id', 'title']
movies = pd.read_csv('/Users/jacquesthibodeau/Desktop/Media/Online Courses/Coding courses/Data Science and Machine Learning with Python/Python Data Science course material/ml-100k/u.item', encoding = "ISO-8859-1", sep='|', names=m_cols, usecols=range(2))

ratings = pd.merge(movies, ratings)

ratings.head()